In [26]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


# gnomAD Queries
## Type1: Explore a particular genomic region
This category include queries that extract information from a region of genome, for example a gene. Becuase gnomAD BigQuery tables utilize [integer range partitioning](https://cloud.google.com/bigquery/docs/creating-integer-range-partitions) they are optimized for this type of queries.

The main requirement to use this feature is to limit queries to a particular region by adding these conditions to the `WHERE` clause:

`WHERE start_position >= X AND start_position <= Y`

Where `[X, Y]` the region of interst. 

You can find values of `X` and `Y` by refering to an external databses. For example following table sumarizes the start and end positions for 4 genes on chromosome 17 extracted from an external resource:

| Gene 	| X 	| Y 	| Source 	|
|:-:	|-	|-	|-	|
| BRCA1 	| 43044295 	| 43125364 	| [link](https://ghr.nlm.nih.gov/gene/BRCA1#location) 	|
| COL1A1 	| 50184096 	| 50201649 	| [link](https://ghr.nlm.nih.gov/gene/COL1A1#location) 	|
| TP53 	| 31094927 	| 31377677 	| [link](https://ghr.nlm.nih.gov/gene/TP53#location) 	|
| NF1 	| 56593699 	| 56595611 	| [link](https://ghr.nlm.nih.gov/gene/NF1#location) 	|

Alternatively you could use the following query that extract the same infomration directly from gnomAD tables. 

In the following example we are using `BRCA1` on `chr17` as example, you could enter your gene of interest to modify all the following queries. Make sure for your gene of interest you are querying the right table (chromosome). If your query returns `NaN` this might be becuase you are querying the wrong table.

Also you could choose which version of gnomAD dataset you'd like to user for all the following queries:
 * `v2_1_1_exomes`
 * `v2_1_1_genomes`
 * `v3_genomes`


In [51]:
# ToDo: Use ipywidgets to get inputs from user
# https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

import ipywidgets as widgets

widgets.Dropdown(
    options=['v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'],
    value='v3_genomes',
    description='gnomAD version:',
    disabled=False,
)

widgets.Dropdown(
    options=['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY'],
    value='chr17',
    description='chromosome:',
    disabled=False,
)

widgets.Text(
    value='BRCA1',
    placeholder='gene_symbol',
    description='Gene Symbol:',
    disabled=False
)

from ipywidgets import interact

def f(x):
    return x

interact(f, x='BRCA1');

Dropdown(description='chromosome:', index=16, options=('chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7',…

In [8]:
# ToDo: Use ipywidgets to get inputs from user
chromosome = 'chr17'
gene_symbol = 'BRCA1'
gnomad_version = 'v3_genomes'

In [10]:
from google.cloud import bigquery

client = bigquery.Client()

def run_query(query):
    query_job = client.query(query)
    result = query_job.to_dataframe(progress_bar_type='tqdm_notebook')
    cost_cents = (query_job.total_bytes_billed / 1024 ** 4) * 500
    print('This query costed (¢): {}'.format(cost_cents))
    return result

In [3]:
query_template = """
SELECT MIN(start_position) AS X, MAX(end_position) AS Y
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table
WHERE EXISTS
  (SELECT 1 FROM UNNEST(main_table.alternate_bases) AS alternate_bases
   WHERE EXISTS (SELECT 1 from alternate_bases.vep WHERE SYMBOL = '{GENE}'))
"""
query = query_template.format(GNOMAD_VER=gnomad_version, CHROM=chromosome, GENE=gene_symbol)

#limits = client.query(query).to_dataframe()
limits = run_query(query)

print(limits)
x = limits.at[0, 'X']
y = limits.at[0, 'Y']


This query costed: 0.0018978118896484375$
          X         Y
0  41191318  41282493


After you found the `[X, Y]` range for your gene of interst you can run *Type1* queries efficiently. Here are a couple of examples:

### Query 1
Find the number of indels and snvs in the region of interest

In [9]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT COUNT(1) AS num, variant_type
FROM (
SELECT DISTINCT 
       start_position,
       reference_bases,
       alternate_bases.alt,
       variant_type,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
)
GROUP BY 2
ORDER BY 1 DESC
"""
query = query_template.format(GNOMAD_VER=gnomad_version, CHROM=chromosome, X=x, Y=y)

summary = run_query(query)
summary.head()


This query costed: 0.00476837158203125¢


,num,variant_type
0,22490,snv
1,2878,indel


Instead of aggregating the results in BigQuery to count the number of each variant type, we could return all rows and process them here. The following query adds a few more columns to the previous query. 

### Query2
A query to retrieve all variants in the region of interest along with `AN` and `AC` values split by gender.

 * `AN`: Total number of alleles in samples
 * `AC`: Alternate allele count for samples

In [11]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT DISTINCT 
       start_position,
       reference_bases,
       AN,
       AN_male,
       AN_female,
       alternate_bases.alt AS alt,
       variant_type,
       alternate_bases.AC AS AC,
       alternate_bases.AC_male AS AC_male,
       alternate_bases.AC_female AS AC_female,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
ORDER BY 1,2
"""
query = query_template.format(GNOMAD_VER=gnomad_version, CHROM=chromosome, X=x, Y=y)

stats_gender = run_query(query)
stats_gender.head()

# ToDo: add some analysis to the returned dataframe


This query costed (¢): 0.00476837158203125


,start_position,reference_bases,AN,AN_male,AN_female,alt,variant_type,num_alts,AC,AC_male,AC_female
0,41191318,T,143350,69482,73868,C,snv,1,1,1,0
1,41191318,T,143350,69482,73868,A,snv,1,3,1,2
2,41191327,A,143350,69468,73882,C,snv,1,5,5,0
3,41191332,G,143310,69444,73866,A,snv,1,1,0,1
4,41191336,C,143268,69420,73848,T,snv,1,2,0,2


Instead of splitting `AN` and `AC` values by gender we can analyze ancestry.

### Query3
A query to retrieve all variants in the region of interest along with `AN` and `AC` values for the following ancestries:
* `afr`: African-American/African ancestry
* `amr`: Latino ancestry
* `eas`: East Asian ancestry
* `nfe`: Non-Finnish European ancestry

In [13]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT DISTINCT 
       start_position,
       reference_bases,
       AN_afr,
       AN_amr,
       AN_eas,
       AN_nfe,
       alternate_bases.alt AS alt,
       variant_type,
       alternate_bases.AC_afr AS AC_afr,
       alternate_bases.AC_amr AS AC_amr,
       alternate_bases.AC_eas AS AC_eas,
       alternate_bases.AC_nfe AS AC_nfe,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
ORDER BY 1,2
"""
query = query_template.format(GNOMAD_VER=gnomad_version, CHROM=chromosome, X=x, Y=y)

stats_ancestry = run_query(query)
stats_ancestry.head()

# ToDo: add some analysis to the returned dataframe


This query costed (¢): 0.00476837158203125


,start_position,reference_bases,AN_afr,AN_amr,AN_eas,AN_nfe,alt,variant_type,AC_afr,AC_amr,AC_eas,AC_nfe
0,41191318,T,42052,13658,3134,64594,C,snv,0,0,0,1
1,41191318,T,42052,13658,3134,64594,A,snv,0,0,0,3
2,41191327,A,42062,13660,3134,64586,C,snv,4,0,0,0
3,41191332,G,42048,13662,3134,64596,A,snv,0,0,0,1
4,41191336,C,42028,13646,3134,64572,T,snv,1,0,0,1


### Query4
Given a region of interest, compute the burden of mutation for the gene along with other summary statistics.

In [27]:
query_template = """
WITH summary_stats AS (
SELECT
  COUNT(1) AS num_variants,
  SUM(ARRAY_LENGTH(alternate_bases)) AS num_alts,  # This data appears to be bi-allelic.
  SUM((SELECT alt.AC FROM UNNEST(alternate_bases) AS alt)) AS sum_AC,
  APPROX_QUANTILES((SELECT alt.AC FROM UNNEST(alternate_bases) AS alt), 10) AS quantiles_AC,
  SUM(AN) AS sum_AN,
  APPROX_QUANTILES(AN, 10) AS quantiles_AN,
  -- Also include some information from Variant Effect Predictor (VEP).
  STRING_AGG(DISTINCT (SELECT annot.symbol FROM UNNEST(alternate_bases) AS alt,
                                                UNNEST(vep) AS annot LIMIT 1), ', ') AS genes
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table
WHERE start_position >= {X} AND start_position <= {Y})
---
--- The resulting quantiles and burden_of_mutation score give a very rough idea of the mutation
--- rate within these particular regions of the genome. This query could be further refined to
--- compute over smaller windows within the regions of interest and/or over different groupings
--- of AC and AN by population.
---
SELECT
  ROUND(({Y} - {X}) / num_variants, 3) AS burden_of_mutation,
  *,
FROM summary_stats
"""
query = query_template.format(GNOMAD_VER=gnomad_version, CHROM=chromosome, X=x, Y=y)

stats = run_query(query)
stats.head()


This query costed: 0.0$


,burden_of_mutation,num_variants,num_alts,sum_AC,quantiles_AC,sum_AN,quantiles_AN,genes
0,3.969,34254,34254,16394200,"[0, 0, 1, 1, 1, 1, 2, 3, 5, 19, 143046]",4625933720,"[274, 122910, 140022, 142438, 142940, 143120, ...","BRCA1, NBR2, NBR1, AC060780.1, AC060780.2, AC0..."


## Type2: Explore an entire chromosome
## Query1
Find 10,000 SNV on chr17 that are more common in women than men, min sample size set to 30.

In [ ]:
%%bigquery result1 --use_rest_api
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC AS AC,
       alternate_bases.AC_female AS AC_f,
       alternate_bases.AC_male AS AC_m,
       ROUND(alternate_bases.AC_female / alternate_bases.AC, 3) AS f_ratio
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE alternate_bases.AC > 30 AND vep.SYMBOL IS NOT NULL
ORDER BY f_ratio DESC
LIMIT 10000


In [12]:
result1.head()

,str_pos,ref,alt,type,gene,f_type,AC,AC_f,AC_m,f_ratio
0,79140505,A,C,snv,AATK,Transcript,41,39,2,0.951
1,79140505,A,C,snv,AATK-AS1,Transcript,41,39,2,0.951
2,33998749,T,TC,ins,AP2B1,Transcript,37,35,2,0.946
3,7221462,T,G,snv,GPS2,Transcript,80,75,5,0.938
4,7221462,T,G,snv,NEURL4,Transcript,80,75,5,0.938


We can condensed the result and only list gene symbols and the number of variants found in the query1.

In [18]:
result1.groupby('gene').count()[['str_pos']].sort_values(by=['str_pos'], ascending=False).head()

,str_pos
gene,
DNAH17,73
CTC-297N7.11,67
RP11-799N11.1,62
RNF213,59
DNAH9,54


## Query2
Find top 1,000 SNV on chr17 that show the most significant differences between male samples of African-American ancestry versus Finnish ancestry

In [ ]:
%%bigquery result2 --use_rest_api
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC_male AS AC_m,
       alternate_bases.AC_fin_male AS AC_fin_m,
       alternate_bases.AC_afr_male AS AC_afr_m,
       ROUND(ABS(alternate_bases.AC_fin_male - alternate_bases.AC_afr_male) / alternate_bases.AC_male, 3) AS fin_afr_diff
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND
      alternate_bases.AC_male > 20 AND alternate_bases.AC_fin_male > 0 AND alternate_bases.AC_afr_male > 0
order by fin_afr_diff DESC
LIMIT 1000

In [22]:
result2.head()

,str_pos,ref,alt,type,gene,f_type,AC_m,AC_fin_m,AC_afr_m,fin_afr_diff
0,76557038,G,A,snv,DNAH17,Transcript,56,53,1,0.929
1,76116856,C,T,snv,TMC6,Transcript,247,228,1,0.919
2,6900268,G,A,snv,ALOX12,Transcript,34,32,1,0.912
3,6900268,G,A,snv,RP11-589P10.5,Transcript,34,32,1,0.912
4,6900268,G,A,snv,RP11-589P10.7,Transcript,34,32,1,0.912


## Query3
Find top 1000 genes with the highest number of SNV on chr17

In [ ]:
%%bigquery result3 --use_rest_api
SELECT gene, count(1) AS num_snv
FROM
(
SELECT DISTINCT 
       start_position AS str_pos,
       alternate_bases.alt AS alt,
       vep.SYMBOL AS gene,
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND alternate_bases.allele_type = 'snv'
)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1000

In [24]:
result3.head()

,gene,num_snv
0,CTC-297N7.11,9589
1,DNAH17,9208
2,RP11-799N11.1,9190
3,RNF213,6561
4,DNAH2,6361
